In [1]:
import imageio.v3 as iio
import requests
import numpy as np
import os
from datetime import datetime
from dotenv import load_dotenv
import sys  
import colorsys
import csv
# TODO: How to compartmentalize these dependencies?

load_dotenv()

True

In [3]:
# Make api call to flickr
FLICKR_API_KEY = os.getenv("FLICKR_API_KEY")

flickr_api_url = 'https://www.flickr.com/services/rest/?method=flickr.photos.search' +\
    '&api_key=' + FLICKR_API_KEY +\
    '&tags=sunset' +\
    '&text=sunset%20landscape' +\
    '&content_type=1' +\
    '&has_geo=1' +\
    '&min_upload_date=2000-01-01' +\
    '&sort=date-posted-asc' +\
    '&extras=date_taken%2C+date_upload%2C+geo%2C+url_o' +\
    '&per_page=250' +\
    '&page=7' +\
    '&format=json&nojsoncallback=1'
# '&is_getty=1' +\

flickr_res = requests.get(flickr_api_url)

if (flickr_res.status_code < 200 or flickr_res.status_code >= 300):
    print('Unsuccessful request. Error code: {}'.format(flickr_res.status_code))
    sys.exit()
else:
    print("Flickr API call was successful.")
    flickr_res = flickr_res.json()
    # print(flickr_res)
    print('Page:')
    print(flickr_res['photos']['page'])
    print('Total number of photos:')
    print(flickr_res['photos']['total'])

Flickr API call was successful.
Page:
7
Total number of photos:
157456


In [4]:
# Process multiple images at once
VISUALCROSSING_API_KEY = os.environ["VISUALCROSSING_API_KEY"]
DATE_TIME_FORMAT_STRING = "%Y-%m-%d %H:%M:%S"
SAMPLE_SCALE = 5

def score_sunset_weightedColors(image):
    sunset_pixel_counter = 0
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            rVal = image[i,j,0]
            gVal = image[i,j,1]  
            bVal = image[i,j,2]
            hVal = colorsys.rgb_to_hls(rVal, gVal, bVal)[0]*360
            if(hVal <= 45): # oranges are weighted less
                sunset_pixel_counter += 0.7
            elif(hVal >= 270): # reds and magentas are weighted fully
                sunset_pixel_counter +=1

    total_image_pixels = image.shape[1] *  image.shape[0]
    sunset_score = ((sunset_pixel_counter / total_image_pixels) * 10)
    sunset_score = round(sunset_score, 3)
    return sunset_score

for i in range(len(flickr_res['photos']['photo'])):
    # Get and load pic from flickr res
    photo_obj = flickr_res['photos']['photo'][i]
    imgUrl = f'https://live.staticflickr.com/{photo_obj["server"]}/{photo_obj["id"]}_{photo_obj["secret"]}_b.jpg'
    pic = iio.imread(imgUrl)
    if len(pic.shape) != 3: # skip over grayscale images
        continue
    
    # GET WEATHER DATA FOR IMAGE
    # nearest hour of sunset for picture
    date_taken = datetime.strptime(photo_obj["datetaken"], DATE_TIME_FORMAT_STRING)
    nearest_sunset_hour = date_taken.hour
    if date_taken.minute > 30:
        nearest_sunset_hour +=  1
    
    # make api call for picture
    visualcrossing_history_api_url = "https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/weatherdata/history?" +\
                            "&aggregateHours=1" +\
                            "&startDateTime=" + str(date_taken.date()) +\
                            "&endDateTime=" + str(date_taken.date())  +\
                            "&unitGroup=metric" +\
                            "&contentType=json" +\
                            "&dayStartTime=" + str(nearest_sunset_hour) + ":00:00" +\
                            "&dayEndTime=" + str(nearest_sunset_hour) + ":01:00" +\
                            "&location=" + photo_obj["latitude"] + "%2C%20" + photo_obj["longitude"] +\
                            "&key=" + VISUALCROSSING_API_KEY
    weather_res = requests.get(visualcrossing_history_api_url)

    if(weather_res.status_code != 200): 
        print('Unsuccessful request. Error code: {}'.format(weather_res.status_code))
        sys.exit()
    else:
        # print('Visual Crossing API call was successful for index ', i)
        weather_res = weather_res.json()
    
    # get weather data of picture
    try:
        nearest_hour_data = weather_res['locations'][photo_obj["latitude"] + ", " + photo_obj["longitude"]]['values'][0]
        humidity = nearest_hour_data['humidity']
        cloud_cover = nearest_hour_data['cloudcover']
        visibility = nearest_hour_data['visibility']
    except KeyError:
        if(weather_res['errorCode'] == -994): # api call quota hit
            print(weather_res)
            break
        else:
            # handles errorCode 999 (weird stuff)
            print(weather_res)
            print(humidity, cloud_cover, visibility)
        
    # skip images with empty weather data
    if(humidity == None or cloud_cover == None or visibility == None):
        print("image" , i, "doesn't have all data:")
        print(humidity, cloud_cover, visibility)
        continue

    # ANALYZE PHOTO
    length = pic.shape[0]
    width = pic.shape[1]
    sys_sample_pic = pic[0:length:SAMPLE_SCALE, 0:width:SAMPLE_SCALE, :]
    calc_score = score_sunset_weightedColors(sys_sample_pic)

    # WRITE TO CSV
    row = [calc_score, humidity, cloud_cover, visibility, "", photo_obj["latitude"], photo_obj["longitude"], date_taken, imgUrl]
    f = open('./data.csv', 'a', newline='')
    writer = csv.writer(f)
    writer.writerow(row)
    f.close()
    print("Image",  i, ", ", imgUrl, ", has been written to csv.")

c:\Python310\lib\colorsys.py:78: RuntimeWarning: overflow encountered in ubyte_scalars
  sumc = (maxc+minc)
c:\Python310\lib\colorsys.py:84: RuntimeWarning: divide by zero encountered in ubyte_scalars
  s = rangec / sumc
c:\Python310\lib\colorsys.py:86: RuntimeWarning: divide by zero encountered in double_scalars
  s = rangec / (2.0-sumc)


Image 0 ,  https://live.staticflickr.com/46/152860557_5b7b4220a3_b.jpg , has been written to csv.
Image 1 ,  https://live.staticflickr.com/47/152860684_19c080342d_b.jpg , has been written to csv.
Image 2 ,  https://live.staticflickr.com/72/152862351_a6be2caa8d_b.jpg , has been written to csv.
Image 3 ,  https://live.staticflickr.com/58/152862496_e0eff33f8a_b.jpg , has been written to csv.
Image 4 ,  https://live.staticflickr.com/52/152870650_83dfe6ebc0_b.jpg , has been written to csv.
Image 5 ,  https://live.staticflickr.com/62/153118799_52595c7f60_b.jpg , has been written to csv.
Image 6 ,  https://live.staticflickr.com/66/153405223_92427172e0_b.jpg , has been written to csv.
Image 7 ,  https://live.staticflickr.com/74/154080214_46b6f54a91_b.jpg , has been written to csv.
image 8 doesn't have all data:
78.79 None None
{'executionTime': -1, 'sessionId': '', 'message': 'No weather stations found within 50.0mi of location -37.385571, -71.377830.', 'errorCode': 999}
78.79 None None
image 

In [21]:
# TESTING CODE
test = flickr_res['photos']['photo'][27]
imgUrl = f'https://live.staticflickr.com/{test["server"]}/{test["id"]}_{test["secret"]}_b.jpg'
sample = iio.imread(imgUrl)
len(sample.shape)

2